In [ ]:
import numpy as np
import qiskit as qk

## Gate Functions

In [ ]:
def init_reg(circ, q, val):
    """Initializes qubit register to value with NOT gates."""
    for i in range(len(q)):
        if val & (1<<i):
            circ.x(q[i])

In [ ]:
def qft(circ, q, n):
    """n-qubit QFT on q in circ."""
    for j in range(n - 1, -1, -1):
        circ.h(q[j])
        for i in range(j):
            circ.cu1(np.pi/float(2**(i + 1)), q[j - (i + 1)], q[j])

def iqft(circ, q, n):
    """n-qubit Inverse QFT on q in circ."""
    for j in range(n):
        for i in range(j - 1, -1, -1):
            circ.cu1(-np.pi/float(2**(i + 1)), q[j - (i + 1)], q[j])
        circ.h(q[j])

In [ ]:
def PhiADDa(circ, a, b, c1=None, c2=None, inverse=False):
    """
    Takes in an n bit number and an n qubit register and
    applies the quantum addition circuit. Optionally takes
    in one or two control qubits. Also, can be inversed to
    make a subtraction circuit.
    """ 
    angles = [0]*len(b)
    for i in range(len(b)):
        if a & (1 << i):
            for j in range(i, len(b)):
                angles[j] += np.pi/2**(j-i)
    
    # Inverse of U1 gates is just a negative theta
    if inverse:
        for i in range(len(angles)):
            angles[i] *= -1
    
    # No controlled bits
    if c1 is None and c2 is None:
        for i in range(len(angles)):
            circ.u1(angles[i], b[i])
    
    # One controlled bit
    if c1 and c2 is None:
        for i in range(len(angles)):
            circ.cu1(angles[i], c1, b[i])
    
    # Two controlled bits
    # Uses sqrt of U1 gates which is just half of theta
    if c1 and c2:
        for i in range(len(angles)):
            circ.cu1(angles[i]/2., c2, b[i])
        circ.cx(c1, c2)
        for i in range(len(angles)):
            circ.cu1(-angles[i]/2., c2, b[i])
        circ.cx(c1, c2)
        for i in range(len(angles)):
            circ.cu1(angles[i]/2., c1, b[i])

In [ ]:
def PhiADDaModN(circ, a, b, c1, c2, ancilla, N, inverse=False):
    """
    Implements the modular adder gate and its inverse.
    (a + b) mod N
    """
    if not inverse:
        PhiADDa(circ, a, b, c1, c2)
        PhiADDa(circ, N, b, inverse=True)

        iqft(circ, b, len(b))
        circ.cx(b[len(b) - 1], ancilla)
        qft(circ, b, len(b))

        PhiADDa(circ, N, b, ancilla)

        PhiADDa(circ, a, b, c1, c2, inverse=True)

        iqft(circ, b, len(b))

        circ.x(b[len(b) - 1])
        circ.cx(b[len(b) - 1], ancilla)
        circ.x(b[len(b) - 1])

        qft(circ, b, len(b))

        PhiADDa(circ, a, b, c1, c2)
        
    else:
        PhiADDa(circ, a, b, c1, c2, inverse=True)
        
        iqft(circ, b, len(b))

        circ.x(b[len(b) - 1])
        circ.cx(b[len(b) - 1], ancilla)
        circ.x(b[len(b) - 1])

        qft(circ, b, len(b))
        
        PhiADDa(circ, a, b, c1, c2)
        
        PhiADDa(circ, N, b, ancilla, inverse=True)
        
        iqft(circ, b, len(b))
        circ.cx(b[len(b) - 1], ancilla)
        qft(circ, b, len(b))
        
        PhiADDa(circ, N, b)
        
        PhiADDa(circ, a, b, c1, c2, inverse=True)

In [ ]:
def CMULTaModN(circ, a, b, c, x, ancilla, N, inverse=False):
    qft(circ, b, len(b))
    
    if inverse:
        bounds = range(len(x) - 1, -1, -1)
    else:
        bounds = range(len(x))
    
    for i in bounds:
        PhiADDaModN(circ, ((2**i)*a) % N, b, c, x[i], ancilla, N, inverse=inverse)
    
    iqft(circ, b, len(b))

In [ ]:
def cswap(circ, c, a, b):
    for i in range(len(a)):
        circ.cx(b[i], a[i])
        circ.ccx(c, a[i], b[i])
        circ.cx(b[i], a[i])

In [ ]:
def modInverse(a, m) : 
    m0 = m 
    y = 0
    x = 1
    
    if (m == 1) : 
        return 0
    
    while (a > 1) : 
        # q is quotient 
        q = a // m 
        
        t = m 
        
        # m is remainder now, process 
        # same as Euclid's algo 
        m = a % m 
        a = t 
        t = y 
        
        # Update x and y 
        y = x - q * y 
        x = t 
    
    # Make x positive 
    if (x < 0) : 
        x = x + m0 
    
    return x

In [ ]:
def cUa(circ, a, c, x, z, ancilla, N):
    CMULTaModN(circ, a, z, c, x, ancilla, N)
    
    cswap(circ, c, x, z)
    
    CMULTaModN(circ, modInverse(a, N), z, c, x, ancilla, N, inverse=True)

## Circuits

In [ ]:
from qiskit.tools.visualization import plot_histogram
sim_backend = qk.BasicAer.get_backend('qasm_simulator')

### Circuit for quantum adder

In [ ]:
b = qk.QuantumRegister(4, 'b')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(b, res_b)

# Add 9 + 4
a = 9
init_reg(circ, b, 4)

qft(circ, b, 4)
PhiADDa(circ, a, b)
iqft(circ, b, 4)

circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

In [ ]:
circ.draw(output='mpl', plot_barriers=False)

### Circuit for quantum adder modulo N

In [ ]:
b = qk.QuantumRegister(4, 'b')
c = qk.QuantumRegister(2, 'c')
anc = qk.QuantumRegister(1, 'anc')
res_b = qk.ClassicalRegister(4, 'res\_b')

circ = qk.QuantumCircuit(c, b, anc, res_b)

# Add 9 + 4 % 11
a = 9
N = 11

init_reg(circ, b, 4)
init_reg(circ, c, 3)

qft(circ, b, len(b))
PhiADDaModN(circ, a, b, c[0], c[1], anc, N)
iqft(circ, b, len(b))

circ.measure(b, res_b)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

### Controlled U-a Circuit

In [ ]:
c = qk.QuantumRegister(1, 'c')
x = qk.QuantumRegister(4, 'x')
z = qk.QuantumRegister(4, 'z')
anc = qk.QuantumRegister(1, 'anc')
res_x = qk.ClassicalRegister(4, 'res\_x')

circ = qk.QuantumCircuit(c, x, z, anc, res_x)

# Multiply 3*5 % 7
a = 5
N = 7
init_reg(circ, x, 3)
init_reg(circ, c, 1)  # C needs to be 1 in order to add, otherwise measurement is x passed through

cUa(circ, a, c[0], x, z, anc[0], N)

circ.measure(x, res_x)

job = qk.execute(circ, sim_backend)
result = job.result()
counts = result.get_counts(circ)
plot_histogram(counts)

Warning: Large circuit, not going to be nice to follow

In [ ]:
circ.draw(output='mpl', plot_barriers=False)